In [2]:
!pip install transformers
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [5]:
import datasets
from datasets import load_dataset,Value, Sequence, Features
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding, AutoModelForMaskedLM
from transformers import default_data_collator
from transformers import DataCollatorForLanguageModeling
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import AutoConfig, AutoModelForMaskedLM, BertForMaskedLM
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
import time,sys,os
import math
import collections



- Traininf data consists of legitimate sequence of REST calls (URI's) to an Alteon device.

- This has been uploaded as HF dataset.
https://huggingface.co/bridge4


In [23]:


def pickle_df_todisk(df, filename=''):
    assert filename,"No filename provided"
    df.to_pickle(filename)
    return "DF saved to file "+filename

def load_pickle_todf(filename=''):
    assert filename,"No filename provided"
    df = pd.read_pickle(filename)
    return df


dfuri = load_pickle_todf(filename='URIanomalygood.pickle')
print (dfuri)
dfuri_ds = Dataset.from_pandas(dfuri)
print (dfuri_ds)
print (dfuri_ds[1])


                                                      URI
0                                                       /
1                                     /webui/default.html
2                                                       /
3                                     /webui/default.html
4                                                       /
...                                                   ...
241620   /monitor/AgSyslogMsgTable?count=1024&props=Index
241621  /monitor/AgSyslogMsgTable?count=50&props=Index...
241622  /monitor?prop=agApplyPending,agSavePending,agS...
241623                                                  /
241624                                /webui/default.html

[241625 rows x 1 columns]
Dataset({
    features: ['URI'],
    num_rows: 241625
})
{'URI': '/webui/default.html'}


- Using distilbert (trained transformer encoder model)  as embeddings model
- Load distilbert and its tokenizer

In [6]:
model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

## load tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print (tokenizer.unk_token_id, tokenizer.unk_token) # 100 [UNK]
print (tokenizer.sep_token, tokenizer.sep_token_id,tokenizer.cls_token, tokenizer.cls_token_id) # [SEP] 102 [CLS] 101


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

100 [UNK]
[SEP] 102 [CLS] 101


**Tokenize Data**

In [7]:

def tokenize_function(examples):

    result = tokenizer(examples["URI"]) # creates input_id and attention mask vectors

    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))] # word_ids represents positional embeddings
    return result


tokenized_datasets = dfuri_ds.map(tokenize_function, batched=True, batch_size=1000)
tokenized_datasets= tokenized_datasets.remove_columns("URI")



Map:   0%|          | 0/241625 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (607 > 512). Running this sequence through the model will result in indexing errors


In [26]:
print (tokenized_datasets)
print (tokenized_datasets[1])
print (dfuri_ds['URI'][1])

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 241625
})
{'input_ids': [101, 1013, 4773, 10179, 1013, 12398, 1012, 16129, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'word_ids': [None, 0, 1, 1, 2, 3, 4, 5, None]}
/webui/default.html


Concat and chunk dataset

In [33]:


chunk_size = 128

def concat_chunk(examples):

    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split concatanated examples by chunk_size
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(concat_chunk, batched=True, batch_size=1000)
lm_datasets



Map:   0%|          | 0/241625 [00:00<?, ? examples/s]

"\nDataset({\n    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],\n    num_rows: 69363\n})\n"

In [47]:
print (lm_datasets[1])
print (tokenizer.decode(lm_datasets[1]["input_ids"]))
print (tokenizer.decode([103])) # mask token
print (tokenizer.decode([101])) # cls token
print (tokenizer.decode([102])) # SEP token

{'input_ids': [4168, 5244, 29336, 16033, 9080, 102, 101, 1013, 9530, 8873, 2290, 1013, 12943, 19620, 9581, 6914, 14074, 102, 101, 1013, 9530, 8873, 2290, 1013, 6131, 21906, 19966, 11149, 21823, 2140, 21084, 3366, 8663, 5104, 102, 101, 1013, 9530, 8873, 2290, 1013, 1044, 2860, 22911, 4215, 16200, 4757, 102, 101, 1013, 9530, 8873, 2290, 1013, 6131, 4168, 5244, 29336, 22747, 9910, 102, 101, 1013, 9530, 8873, 2290, 1013, 11867, 9153, 3215, 21906, 2226, 21823, 24458, 3468, 102, 101, 1013, 9530, 8873, 2290, 1013, 2038, 9148, 10649, 2378, 14876, 9153, 2618, 102, 101, 1013, 9530, 8873, 2290, 1013, 1044, 21724, 6444, 5332, 4371, 102, 101, 1013, 9530, 8873, 2290, 1013, 2524, 10521, 22287, 8528, 102, 101, 1013, 9530, 8873, 2290, 1013, 2524, 10521, 22287, 8528, 102, 101, 1013, 9530, 8873], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

**Whole word masking Collator**

In [49]:


import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.13


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id # 103 which is the mask token [MASK]
        feature["labels"] = new_labels

    return default_data_collator(features)



samples = [lm_datasets[i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)
print (batch["input_ids"][0], "\n", tokenizer.decode(batch["input_ids"][0]), "\n", batch["labels"][0])




tensor([  101,   103,   102,   101,   103,  4773, 10179,  1013,   103,  1012,
        16129,   102,   101,  1013,   102,   101,  1013,  4773, 10179,  1013,
        12398,  1012, 16129,   102,   101,  1013,   102,   101,  1013,  4773,
        10179,  1013,   103,  1012, 16129,   102,   101,  1013,  6398,  1013,
         2897,   102,   101,  1013,  6398,  1013,  2291,   102,   101,  1013,
          103,   103,  7484,  8043,  6299,   102,   101,  1013,   102,   101,
         1013,  4773, 10179,  1013, 12398,  1012, 16129,   102,   101,  1013,
          102,   101,  1013,  4773, 10179,  1013, 12398,  1012, 16129,   102,
          101,  1013,   102,   101,  1013,   103,   103,  1013, 12398,  1012,
        16129,   102,   101,  1013,  9530,  8873,  2290,  1013, 11867,  9153,
         3215, 21906,  2226, 21823, 24458,  3468,   102,   101,  1013,  9530,
         8873,  2290,  1013,   103,   103,   103,   103,   103,   103,   103,
          102,   101,  1013,   103,   103,   103,  1013,  6131])

In [ ]:
batch_size = 32
epochs = 4

train_loader = DataLoader(lm_datasets,shuffle=True,batch_size=batch_size, collate_fn=whole_word_masking_data_collator)
optimizer = AdamW(model.parameters(), lr=0.00005)
print ("Is GPU available ? : ", torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


########### Training Loop ########################################

def trainDistelbertMask(net, optimizer, device, epochs, train_loader,
                        clipping =0, Xkey='input_ids',attnkey = 'attention_mask', ykey='labels'):

    trainperplex = []
    losses   = []

    starttime = time.time()

    net.to(device)
    for epochi in range(epochs):

        net.train() #flag to turn on train

        batchAcc = []
        batchLoss = []

        print ("Time ( in secs) elapsed since start of training : ",time.time()-starttime)

        for batchidx, batch in enumerate(train_loader):

            net.train()


            X = batch[Xkey]

            attn_mask = batch[attnkey]

            y = batch[ykey]

            X = X.to(device)
            attn_mask = attn_mask.to(device)
            y = y.to(device)

            outputs = net(X,attention_mask= attn_mask,labels = y)

            loss = outputs.loss

            optimizer.zero_grad()

            loss.backward()

            if clipping > 0:
                torch.nn.utils.clip_grad_norm_(net.parameters(), clipping)

            optimizer.step()

            loss = loss.cpu()

            batchLoss.append(loss.item())

            print ('At Batchidx %d in epoch %d: '%(batchidx,epochi), "loss is %f "% (loss.item()))



            ##### end of batch loop######


        tmpmeanbatchloss = np.mean(batchLoss)
        try:
            perplexity = math.exp(tmpmeanbatchloss)
        except OverflowError:
            perplexity = float("inf")

        losses.append(tmpmeanbatchloss)
        trainperplex.append(perplexity)



        print("##Epoch %d averaged batch training perplexity is %f and loss is %f"%(epochi,perplexity,
                                                                                          tmpmeanbatchloss))


    print ("Time ( in secs) elapsed since start of training : ",time.time()-starttime) # final time once all training done

    return trainperplex, losses


trainperplex, losses = trainDistelbertMask(model, optimizer, device, epochs, train_loader)



**Testing inference on trained model**

In [ ]:
from transformers import pipeline

# Loading saved model that was trained previous cell. The trained model had been uploaded to huggingface https://huggingface.co/bridge4

model = AutoModelForMaskedLM.from_pretrained("4epochdistilbert_uri_domainadapt")
urltext = "/[MASK]"
mask_filler = pipeline(
    "fill-mask", model=model,
    tokenizer = tokenizer
)
preds = mask_filler(urltext)
print (preds)
for pred in preds:
    print(f">>> {pred['sequence']}")

"""
>>> / system
>>> / network
>>> / html
>>> / reporter
"""